## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization
from transformers import BertConfig, BertTokenizer, TFBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-tf_model.h5'
VOCAB_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-vocab.txt'
model_path_list = glob.glob('/kaggle/input/140-googleq-a-train-3fold-bert-base-unc-raw-huface/' + '*1.h5')
model_path_list += glob.glob('/kaggle/input/140-googleq-a-train-3fold-bert-base-unc-raw-huface/' + '*2.h5')
model_path_list += glob.glob('/kaggle/input/140-googleq-a-train-3fold-bert-base-unc-raw-huface/' + '*3.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/140-googleq-a-train-3fold-bert-base-unc-raw-huface/model_fold_1.h5
/kaggle/input/140-googleq-a-train-3fold-bert-base-unc-raw-huface/model_fold_2.h5
/kaggle/input/140-googleq-a-train-3fold-bert-base-unc-raw-huface/model_fold_3.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = BertTokenizer.from_pretrained(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
bert_config = BertConfig()
bert_config.output_hidden_states=False

def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_model = TFBertModel.from_pretrained(BERT_PATH, config=bert_config)
    sequence_output, pooled_output = bert_model([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.939659,0.636260,0.209095,0.517714,0.591920,0.547158,0.671477,0.671813,0.671853,...,0.905061,0.897970,0.573826,0.963302,0.965611,0.799106,0.052498,0.067196,0.866889,0.904376
1,46,0.879376,0.556844,0.005063,0.758570,0.805026,0.944186,0.543537,0.456715,0.061984,...,0.738701,0.948977,0.628432,0.971489,0.979108,0.870339,0.892675,0.122915,0.136263,0.900138
2,70,0.926201,0.690540,0.029586,0.759330,0.872744,0.915747,0.599228,0.514503,0.191895,...,0.881730,0.930254,0.602000,0.965038,0.970684,0.866008,0.074548,0.059125,0.857847,0.916206
3,132,0.898507,0.467175,0.006385,0.748808,0.740768,0.936044,0.519524,0.411754,0.094934,...,0.698404,0.961085,0.684382,0.970666,0.985671,0.915629,0.853219,0.135318,0.413749,0.909327
4,200,0.922178,0.461648,0.032658,0.851897,0.763758,0.875704,0.620653,0.546095,0.121948,...,0.676788,0.913535,0.635394,0.969855,0.968604,0.857898,0.182692,0.082484,0.849316,0.910053


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.903008,0.611739,0.035204,0.726244,0.796666,0.866781,0.569608,0.490571,0.236056,...,0.788594,0.938301,0.658700,0.966365,0.974100,0.875843,0.499792,0.127062,0.555944,0.910064
std,2812.670060,0.032711,0.112292,0.059862,0.091441,0.090662,0.110978,0.050949,0.086811,0.191776,...,0.075223,0.020318,0.045626,0.008480,0.009715,0.033115,0.317790,0.054860,0.274882,0.016719
min,39.000000,0.807172,0.385040,0.003750,0.318803,0.283226,0.290298,0.465791,0.342701,0.013776,...,0.599920,0.858751,0.522770,0.925688,0.925712,0.774757,0.006615,0.011426,0.037468,0.852630
25%,2572.000000,0.883091,0.513983,0.007904,0.682000,0.750101,0.855799,0.529842,0.429129,0.085801,...,0.730267,0.926997,0.626064,0.961322,0.968186,0.855824,0.163935,0.091113,0.323738,0.898956
50%,5093.000000,0.901960,0.607658,0.012758,0.731566,0.803742,0.911243,0.564760,0.469834,0.162207,...,0.792468,0.941811,0.656443,0.967644,0.976014,0.879935,0.573398,0.126785,0.556520,0.910767
75%,7482.000000,0.925914,0.699753,0.029963,0.783113,0.855117,0.930489,0.602018,0.532528,0.349282,...,0.851245,0.953179,0.691118,0.971975,0.981885,0.901350,0.788282,0.161849,0.797590,0.922114
max,9640.000000,0.975986,0.875277,0.531363,0.956027,0.970830,0.966792,0.735049,0.756011,0.851834,...,0.936907,0.976400,0.789906,0.984518,0.989328,0.951394,0.947927,0.277912,0.991073,0.955519
